# Visualizing Chipotle's Data

This time we are going to pull data directly from the internet.
Special thanks to: https://github.com/justmarkham for sharing the dataset and materials.

### Step 1. Import the necessary libraries

In [2]:
import pandas as pd
import matplotlib.pyplot as plt
from collections import Counter

# set this so the graphs open internally
%matplotlib inline

In [3]:
import tensorflow as tf
print(f"TensorFlow version: {tf.__version__}")
print(f"GPUs disponibles: {len(tf.config.list_physical_devices('GPU'))}")
print(f"GPU support: {tf.test.is_gpu_available()}")

2025-07-01 19:15:33.733357: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-07-01 19:15:33.733385: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-07-01 19:15:33.734411: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-07-01 19:15:33.739104: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


TensorFlow version: 2.15.0
GPUs disponibles: 1
Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.
GPU support: True


2025-07-01 19:15:34.885300: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2025-07-01 19:15:34.914798: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2025-07-01 19:15:34.915009: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-

In [4]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("Test").getOrCreate()
print(f"PySpark version: {spark.version}")
spark.range(10).show()
spark.stop()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/07/01 19:15:36 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


PySpark version: 3.5.0
+---+
| id|
+---+
|  0|
|  1|
|  2|
|  3|
|  4|
|  5|
|  6|
|  7|
|  8|
|  9|
+---+



In [5]:
import os
import sys

# Verificar variables de entorno importantes
env_vars = ['JAVA_HOME', 'PYSPARK_PYTHON', 'PYSPARK_DRIVER_PYTHON']
for var in env_vars:
    value = os.environ.get(var, 'No configurada')
    print(f"🔧 {var}: {value}")

print(f"🐍 Python version: {sys.version}")


🔧 JAVA_HOME: /usr/lib/jvm/java-11-openjdk-amd64
🔧 PYSPARK_PYTHON: python3
🔧 PYSPARK_DRIVER_PYTHON: python3
🐍 Python version: 3.11.0rc1 (main, Aug 12 2022, 10:02:14) [GCC 11.2.0]


In [6]:
import os
from pathlib import Path

def setup_working_directory():
    """
    Configura el directorio de trabajo automáticamente según la ubicación del notebook.
    """
    try:
        # Esto obtiene la ruta del notebook actual
        notebook_path = Path.cwd()  # Esto funciona en Jupyter
        os.chdir(notebook_path)
        print(f"📁 Cambiado automáticamente a: {notebook_path}")
    except Exception as e:
        print(f"⚠️ Error: {e}")
        print(f"📁 Manteniéndose en: {os.getcwd()}")

# Llama a la función sin parámetros
setup_working_directory()

📁 Cambiado automáticamente a: /tf


In [7]:
os.chdir('/tf/pyspark_exercises/07_Visualization/Chipotle/')


### Step 2. Import the dataset from this [address](https://raw.githubusercontent.com/justmarkham/DAT8/master/data/chipotle.tsv). 

### Step 3. Assign it to a variable called chipo.

In [10]:
from pyspark.sql import SparkSession


spark = SparkSession.builder\
                    .appName('chipo')\
                    .getOrCreate()

In [8]:
import os

os.getcwd()

'/tf/pyspark_exercises/07_Visualization/Chipotle'

In [11]:
import requests
url = 'https://raw.githubusercontent.com/justmarkham/DAT8/master/data/chipotle.tsv'

get_url = requests.get(url)


with open('data.csv', 'w', encoding='utf-8') as f:

    f.writelines(get_url.text)



In [31]:

def file_resolver(path):

    return str(Path(path).resolve())


In [33]:
chipo = spark.read.csv(file_resolver('data.csv'), header=True, sep='\t')

chipo.show(5)

+--------+--------+--------------------+--------------------+----------+
|order_id|quantity|           item_name|  choice_description|item_price|
+--------+--------+--------------------+--------------------+----------+
|       1|       1|Chips and Fresh T...|                NULL|    $2.39 |
|       1|       1|                Izze|        [Clementine]|    $3.39 |
|       1|       1|    Nantucket Nectar|             [Apple]|    $3.39 |
|       1|       1|Chips and Tomatil...|                NULL|    $2.39 |
|       2|       2|        Chicken Bowl|[Tomatillo-Red Ch...|   $16.98 |
+--------+--------+--------------------+--------------------+----------+
only showing top 5 rows



In [35]:
df = pd.read_csv(file_resolver('data.csv'), header=0, sep='\t')

### Step 4. See the first 10 entries

In [34]:
chipo.show(10)

+--------+--------+--------------------+--------------------+----------+
|order_id|quantity|           item_name|  choice_description|item_price|
+--------+--------+--------------------+--------------------+----------+
|       1|       1|Chips and Fresh T...|                NULL|    $2.39 |
|       1|       1|                Izze|        [Clementine]|    $3.39 |
|       1|       1|    Nantucket Nectar|             [Apple]|    $3.39 |
|       1|       1|Chips and Tomatil...|                NULL|    $2.39 |
|       2|       2|        Chicken Bowl|[Tomatillo-Red Ch...|   $16.98 |
|       3|       1|        Chicken Bowl|[Fresh Tomato Sal...|   $10.98 |
|       3|       1|       Side of Chips|                NULL|    $1.69 |
|       4|       1|       Steak Burrito|[Tomatillo Red Ch...|   $11.75 |
|       4|       1|    Steak Soft Tacos|[Tomatillo Green ...|    $9.25 |
|       5|       1|       Steak Burrito|[Fresh Tomato Sal...|    $9.25 |
+--------+--------+--------------------+-----------

In [36]:
df.head(10)

,order_id,quantity,item_name,choice_description,item_price
0,1,1,Chips and Fresh Tomato Salsa,NaN,$2.39
1,1,1,Izze,[Clementine],$3.39
2,1,1,Nantucket Nectar,[Apple],$3.39
3,1,1,Chips and Tomatillo-Green Chili Salsa,NaN,$2.39
4,2,2,Chicken Bowl,"[Tomatillo-Red Chili Salsa (Hot), [Black Beans...",$16.98
5,3,1,Chicken Bowl,"[Fresh Tomato Salsa (Mild), [Rice, Cheese, Sou...",$10.98
6,3,1,Side of Chips,NaN,$1.69
7,4,1,Steak Burrito,"[Tomatillo Red Chili Salsa, [Fajita Vegetables...",$11.75
8,4,1,Steak Soft Tacos,"[Tomatillo Green Chili Salsa, [Pinto Beans, Ch...",$9.25
9,5,1,Steak Burrito,"[Fresh Tomato Salsa, [Rice, Black Beans, Pinto...",$9.25


### Step 5. Create a histogram of the top 5 items bought

### Step 6. Create a scatterplot with the number of items orderered per order price
#### Hint: Price should be in the X-axis and Items ordered in the Y-axis

### Step 7. BONUS: Create a question and a graph to answer your own question.